Python script to create a many-body expansion calculation dataset with qcmanybody
Usage: modify the setting lines below and run `python mbe_dataset.py`

In [ ]:
import qcportal as ptl
from qcportal.molecules import Molecule
from qcportal.record_models import RecordStatusEnum
import numpy as np
from qcportal.manybody import ManybodySpecification
import glob

Settings

In [ ]:
client = ptl.PortalClient("http://127.0.0.1:7777")

In [ ]:
GEOM_FILE = 'test-5.xyz' # A stacked xyz file containing multiple geoms of water clusters

In [ ]:
N = 3

In [ ]:
DELETE_SCRATCH = True # if false keeping the scratch files for debugging purpose

In [ ]:
DATASET_NAME = 'W3_MBE_test'

In [ ]:
DATASET_DESCRIPTION = 'MBE calculations of water trimers, test dataset run'

In [ ]:
# specification
sp_spec = {
    "program": "psi4",
    "driver": "energy",
    "method": "b3lyp",
    "basis": "6-31G*",
    "keywords": {"e_convergence": 1e-10, "d_convergence": 1e-10},
}

In [ ]:
mb_spec = ManybodySpecification(
    bsse_correction=['vmfc'],
    levels={1: sp_spec, 2: sp_spec, 3: sp_spec},
)

In [ ]:
SPEC_NAME = 'b3lyp/6-31G*/vmfc'

In [ ]:
ENTRY_PRE = 'trimer_'

End Settings

In [ ]:
i = -1

In [ ]:
row_length = 12*N # no. of elements for a geom row of a structure

In [ ]:
geom_blank = [" "]*row_length
geom = [geom_blank]

In [ ]:
j = 0

In [ ]:
with open(GEOM_FILE,'r') as Inp:
    for line in Inp:
        line_sp = line.split()
        if len(line_sp) == 1:
            i += 1
            j = 0
            if i != 0:
                geom = np.vstack([geom,geom_blank])
        elif len(line_sp) == 4:
            if i == 0:
                geom[j:j+4] = line_sp
            else:
                geom[i,j:j+4] = line_sp
            j += 4

In [ ]:
ds_w3_mbe = client.add_dataset("manybody",
                        name=DATASET_NAME,
                        description=DATASET_DESCRIPTION)

In [ ]:
# add water trimers as entries
for ii in range(len(geom)):
    with open('tmp-geom'+str(ii)+'.in', 'w+') as GeomInp:
        for j in range(0,row_length,4):
            GeomInp.write("{} {} {} {}\n".format(geom[ii,j],float(geom[ii,j+1]),float(geom[ii,j+2]),float(geom[ii,j+3])))
            if ((j+4)%12 == 0 and ((j+4) < row_length)):
                GeomInp.write("--\n")
        GeomInp.seek(0)
        mol_geom = GeomInp.read()
        mol = Molecule.from_data(mol_geom)
    entry_name = ENTRY_PRE+str(ii)
    ds_w3_mbe.add_entry(name=entry_name, initial_molecule=mol)

In [ ]:
# Actually attach the specification and entry to the dataset
ds_w3_mbe.add_specification(name=SPEC_NAME, specification=mb_spec)

In [ ]:
# Then from there, you can submit, etc
ds_w3_mbe.submit()

In [ ]:
# clean scratch
if DELETE_SCRATCH:
    for f in glob.glob("tmp-geom*.in"):
        os.remove(f)

ds_w3_mbe.print_status()

get a record and show the output
rec = ds_w3_mbe.get_record('trimer_1', 'b3lyp/6-31G*/vmfc')
print(rec.stdout)

calculate the MBE errors
FULL_BODY = 3
MAX_NBODY = 3

full_body_key = 'VMFC-CORRECTED TOTAL ENERGY THROUGH '+str(FULL_BODY)+'-BODY'
full_body_E = rec.results['results'][full_body_key]

MBE_err = [] # list of MBE errors

for i in range(MAX_NBODY - 1):
   key = 'VMFC-CORRECTED TOTAL ENERGY THROUGH '+str(i+1)+'-BODY'
   MBE_err.append(rec.results['results'][key] - full_body_E)

print(MBE_err)